In [2]:
import os
import sys


# Note that the path to spark home could be different for you. This one is for HortonWorks Data Platform
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"

# The version for py4j could be different for you
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.10:0.4.1 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell'
# com.databricks:spark-avro_2.11:3.2.0
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.6.0-spark2.3-s_2.11,com.databricks:spark-xml_2.11:0.6.0,com.databricks:spark-avro_2.11:3.2.0 pyspark-shell'

In [3]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

# Remove the unneccessary text

In [64]:
lines = sc.textFile("/data/loan_stats_3c.csv")

In [65]:
lines1 = lines.filter(lambda line: not (line.startswith('Notes offered by Prospectus') or line.startswith('Total amount funded in policy code') ) )

In [32]:
lines1.take(1)

['"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","open_il_12m","open_il_24m","mths_since_rcnt_il","total_bal_il","il_util","open_rv

In [33]:
lines2 = lines1.repartition(1)

In [35]:
lines2.saveAsTextFile('clean_loan_stats_3c.csv', )

In [36]:
!hadoop fs -ls clean_loan_stats_3c.csv/

Found 2 items
-rw-r--r--   3 sandeepgiri9034 sandeepgiri9034          0 2019-09-03 17:39 clean_loan_stats_3c.csv/_SUCCESS
-rw-r--r--   3 sandeepgiri9034 sandeepgiri9034  186874034 2019-09-03 17:39 clean_loan_stats_3c.csv/part-00000


In [34]:
!hadoop fs -rmr clean_loan_stats_3c.csv/

rmr: DEPRECATED: Please use 'rm -r' instead.
19/09/03 17:39:22 INFO fs.TrashPolicyDefault: Moved: 'hdfs://cxln1.c.thelab-240901.internal:8020/user/sandeepgiri9034/clean_loan_stats_3c.csv' to trash at: hdfs://cxln1.c.thelab-240901.internal:8020/user/sandeepgiri9034/.Trash/Current/user/sandeepgiri9034/clean_loan_stats_3c.csv1567532362027


In [37]:
!hadoop fs -tail clean_loan_stats_3c.csv/part-00000

"2","18","0","0","0","0","97.6","100","0","0","96582","73598","8500","83482","","","","","","","","","","","","N","","","","","","","","","","","","","","","N","","","","","",""
"","","10000","10000","9975"," 36 months"," 19.20%","367.58","D","D3","","10+ years","MORTGAGE","46000","Verified","Jan-2014","Fully Paid","n","","  Borrower added on 12/04/13 > I will like a loan to fix  our cars and paid our house property tax<br>","other","Other","950xx","CA","22.78","1","Feb-2000","0","9","","6","0","11325","70.8%","22","f","0.00","0.00","13232.50333937","13199.42","10000.00","3232.50","0.0","0.0","0.0","Jan-2017","367.2","","Nov-2017","0","","1","Individual","","","","0","0","57477","","","","","","","","","","","","16000","","","","4","11495","2175","83.9","0","0","93","166","4","4","2","4","","13","9","0","2","2","2","11","3","4","17","2","6","0","0","0","2","90.9","50","0","0","81470","57477","13500","65470","","","","","","","","","","","","N","","","","","","","","","","","","","","",

# End of removing the unneccessary text

In [66]:
loan_stats = spark.read.csv("/data/clean_loan_stats_3c.csv", header=True, inferSchema=True)

In [67]:
loan_stats.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: integer (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: doub

In [6]:
loan_stats.show()

+----+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+--------+-----------+----------+----+----+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+----------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+-------

In [64]:
loan_stats.count()

235629

In [66]:
len(loan_stats.columns)

144

In [68]:
loan_stats

DataFrame[id: string, member_id: string, loan_amnt: int, funded_amnt: int, funded_amnt_inv: int, term: string, int_rate: string, installment: double, grade: string, sub_grade: string, emp_title: string, emp_length: string, home_ownership: string, annual_inc: double, verification_status: string, issue_d: string, loan_status: string, pymnt_plan: string, url: string, desc: string, purpose: string, title: string, zip_code: string, addr_state: string, dti: double, delinq_2yrs: int, earliest_cr_line: string, inq_last_6mths: int, mths_since_last_delinq: int, mths_since_last_record: int, open_acc: int, pub_rec: int, revol_bal: int, revol_util: string, total_acc: int, initial_list_status: string, out_prncp: double, out_prncp_inv: double, total_pymnt: double, total_pymnt_inv: double, total_rec_prncp: double, total_rec_int: double, total_rec_late_fee: double, recoveries: double, collection_recovery_fee: double, last_pymnt_d: string, last_pymnt_amnt: double, next_pymnt_d: string, last_credit_pull_

In [7]:
kv_loan = loan_stats.rdd.flatMap(lambda r: r.asDict().items())

In [8]:
# print Missing Values
kv_loan.filter(lambda kv: kv[1] is None).mapValues(lambda x: 1).reduceByKey(lambda x,y: x+y).collect()


[('verification_status_joint', 235629),
 ('sec_app_revol_util', 235629),
 ('hardship_loan_status', 235124),
 ('avg_cur_bal', 6),
 ('id', 235629),
 ('sec_app_num_rev_accts', 235629),
 ('sec_app_mths_since_last_major_derog', 235629),
 ('settlement_date', 231362),
 ('percent_bc_gt_75', 2557),
 ('sec_app_mort_acc', 235629),
 ('orig_projected_additional_accrued_interest', 235228),
 ('mths_since_last_delinq', 115881),
 ('revol_bal_joint', 235629),
 ('sec_app_chargeoff_within_12_mths', 235629),
 ('url', 235629),
 ('deferral_term', 235124),
 ('settlement_status', 231362),
 ('sec_app_inq_last_6mths', 235629),
 ('settlement_term', 231362),
 ('mths_since_last_major_derog', 169151),
 ('revol_util', 125),
 ('annual_inc_joint', 235629),
 ('il_util', 235629),
 ('inq_last_12m', 235629),
 ('hardship_dpd', 235124),
 ('debt_settlement_flag_date', 231362),
 ('dti_joint', 235629),
 ('open_act_il', 235629),
 ('inq_fi', 235629),
 ('sec_app_open_act_il', 235629),
 ('num_tl_120dpd_2m', 7860),
 ('mths_since_rec

In [72]:
loanstatus.take(10)

[Row(loan_status='Charged Off'),
 Row(loan_status='Fully Paid'),
 Row(loan_status='Fully Paid'),
 Row(loan_status='Charged Off'),
 Row(loan_status='Current'),
 Row(loan_status='Fully Paid'),
 Row(loan_status='Fully Paid'),
 Row(loan_status='Charged Off'),
 Row(loan_status='Fully Paid'),
 Row(loan_status='Charged Off')]

In [74]:
loanstatus_vals = loanstatus.map(lambda r: (r['loan_status'], 1))

In [75]:
loanstatus_vals.countByKey()

defaultdict(int,
            {'Charged Off': 41201,
             'Fully Paid': 182794,
             'Current': 11097,
             'Late (16-30 days)': 94,
             'Late (31-120 days)': 307,
             'Default': 6,
             'In Grace Period': 130})

In [11]:
loan_stats.select('loan_status').show()

+-----------+
|loan_status|
+-----------+
|Charged Off|
| Fully Paid|
| Fully Paid|
|Charged Off|
|    Current|
| Fully Paid|
| Fully Paid|
|Charged Off|
| Fully Paid|
|Charged Off|
| Fully Paid|
| Fully Paid|
|Charged Off|
|    Current|
| Fully Paid|
| Fully Paid|
| Fully Paid|
| Fully Paid|
| Fully Paid|
| Fully Paid|
+-----------+
only showing top 20 rows



In [68]:
data_with_loanstatus = loan_stats.filter("loan_status == 'Fully Paid' or loan_status == 'Charged Off' ")

In [6]:
Dataset_withBoolTarget = data_with_loanstatus.replace(['Fully Paid', 'Charged Off'], [False, True], 'loan_status')

In [9]:
r = Dataset_withBoolTarget.select('loan_status')

In [10]:
r.rdd.map(lambda row: (row['loan_status'], 1)).reduceByKey(lambda x,y: x+y).collect()

[('false', 182794), ('true', 41201)]

In [40]:
# # remove the columns with 95% of the data is not null
# # 95% of 235631 is 223849 so we set the threshold according to that
# dataset = Dataset_withBoolTarget.dropna(thresh = 223849) #223849 is minimum number of non-NA values


In [46]:
Dataset_withBoolTarget.columns

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_

In [69]:
dataset = data_with_loanstatus

In [90]:
# we have selected 15 features 
features = ['funded_amnt','annual_inc','home_ownership','grade',
            "last_pymnt_amnt", "mort_acc", "pub_rec", "int_rate", "open_acc","num_actv_rev_tl",
            "mo_sin_rcnt_rev_tl_op","mo_sin_old_rev_tl_op",
            "avg_cur_bal","acc_open_past_24mths",'loan_status'] 

Final_data = dataset[features] # 17 features with target var

In [91]:
Final_data.show()

+-----------+----------+--------------+-----+---------------+--------+-------+--------+--------+---------------+---------------------+--------------------+-----------+--------------------+-----------+
|funded_amnt|annual_inc|home_ownership|grade|last_pymnt_amnt|mort_acc|pub_rec|int_rate|open_acc|num_actv_rev_tl|mo_sin_rcnt_rev_tl_op|mo_sin_old_rev_tl_op|avg_cur_bal|acc_open_past_24mths|loan_status|
+-----------+----------+--------------+-----+---------------+--------+-------+--------+--------+---------------+---------------------+--------------------+-----------+--------------------+-----------+
|      10400|   58000.0|      MORTGAGE|    A|         321.08|       1|      0|   6.99%|      17|              9|                    1|                 290|       9536|                   7|Charged Off|
|      15000|   78000.0|          RENT|    C|       12017.81|       0|      0|  12.39%|       6|              4|                    1|                 244|      29828|                   5| Fully P

In [15]:
Final_data.select('int_rate').show()

+--------+
|int_rate|
+--------+
|   6.99%|
|  12.39%|
|  13.66%|
|  13.66%|
|  15.59%|
|  13.66%|
|  14.31%|
|  11.99%|
|  17.86%|
|  11.44%|
|  11.44%|
|  11.99%|
|  15.99%|
|  10.49%|
|  10.49%|
|  14.99%|
|  15.99%|
|  10.49%|
|  14.31%|
|  12.99%|
+--------+
only showing top 20 rows



In [92]:
from pyspark.sql import Row

In [93]:
def replacepercent(row):
    d = row.asDict()
    v = float(d['int_rate'][:-1])
    d['int_rate'] = v
    return Row(**d)

In [73]:
# Test the function
rows = Final_data.rdd.take(10)
replacepercent(rows[0])

Row(acc_open_past_24mths=7, annual_inc=58000.0, avg_cur_bal=9536, funded_amnt=10400, grade='A', home_ownership='MORTGAGE', int_rate=6.99, last_pymnt_amnt=321.08, loan_status='Charged Off', mo_sin_old_rev_tl_op=290, mo_sin_rcnt_rev_tl_op=1, mort_acc=1, num_actv_rev_tl=9, open_acc=17, pub_rec=0)

In [94]:
Final_data = Final_data.rdd.map(replacepercent).toDF()

In [16]:
Final_data.show()

+--------------------+----------+-----------+-----------+-----+--------------+--------+---------------+-----------+--------------------+---------------------+--------+---------------+--------+-------+
|acc_open_past_24mths|annual_inc|avg_cur_bal|funded_amnt|grade|home_ownership|int_rate|last_pymnt_amnt|loan_status|mo_sin_old_rev_tl_op|mo_sin_rcnt_rev_tl_op|mort_acc|num_actv_rev_tl|open_acc|pub_rec|
+--------------------+----------+-----------+-----------+-----+--------------+--------+---------------+-----------+--------------------+---------------------+--------+---------------+--------+-------+
|                   7|   58000.0|       9536|      10400|    A|      MORTGAGE|    6.99|         321.08|       true|                 290|                    1|       1|              9|      17|      0|
|                   5|   78000.0|      29828|      15000|    C|          RENT|   12.39|       12017.81|      false|                 244|                    1|       0|              4|       6|    

In [26]:
Final_data.printSchema()

root
 |-- funded_amnt: integer (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- last_pymnt_amnt: double (nullable = true)
 |-- mort_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- num_actv_rev_tl: integer (nullable = true)
 |-- mo_sin_rcnt_rev_tl_op: integer (nullable = true)
 |-- mo_sin_old_rev_tl_op: integer (nullable = true)
 |-- avg_cur_bal: integer (nullable = true)
 |-- acc_open_past_24mths: integer (nullable = true)
 |-- loan_status: string (nullable = true)



In [17]:
def df_value_counts(df, col):
    return df.rdd.map(lambda row: (row[col], 1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: -x[1]).collect()

In [18]:
df_value_counts(Final_data, 'home_ownership')

[('MORTGAGE', 113148), ('RENT', 88933), ('OWN', 21913), ('ANY', 1)]

In [93]:
df_value_counts(Final_data, 'grade')

[('C', 62823),
 ('B', 60028),
 ('D', 39596),
 ('A', 35881),
 ('E', 18370),
 ('F', 5765),
 ('G', 1532)]

In [95]:
def replace_row(row, repdict):
    d = row.asDict()
    for k, torep in repdict.items():
        if k in d:
            d[k] = torep[d[k]]
    return Row(**d)

def df_replace(df, repdict):
    return df.rdd.map(lambda r: replace_row(r, repdict))

In [96]:
Final_data = df_replace(Final_data, { 'grade': {'A':7,'B':6,'C':5,'D':4,'E':3,'F':2,'G':1}, 'home_ownership': {"MORTGAGE":4,"RENT":3,"OWN":2,"ANY":1}, 'loan_status': {'Fully Paid':0.0, 'Charged Off':1.0}})

In [97]:
Final_data = Final_data.toDF()

In [78]:
Final_data.show()

+--------------------+----------+-----------+-----------+-----+--------------+--------+---------------+-----------+--------------------+---------------------+--------+---------------+--------+-------+
|acc_open_past_24mths|annual_inc|avg_cur_bal|funded_amnt|grade|home_ownership|int_rate|last_pymnt_amnt|loan_status|mo_sin_old_rev_tl_op|mo_sin_rcnt_rev_tl_op|mort_acc|num_actv_rev_tl|open_acc|pub_rec|
+--------------------+----------+-----------+-----------+-----+--------------+--------+---------------+-----------+--------------------+---------------------+--------+---------------+--------+-------+
|                   7|   58000.0|       9536|      10400|    7|             4|    6.99|         321.08|          1|                 290|                    1|       1|              9|      17|      0|
|                   5|   78000.0|      29828|      15000|    5|             3|   12.39|       12017.81|          0|                 244|                    1|       0|              4|       6|    

In [79]:
Final_data.printSchema()

root
 |-- acc_open_past_24mths: long (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- avg_cur_bal: long (nullable = true)
 |-- funded_amnt: long (nullable = true)
 |-- grade: long (nullable = true)
 |-- home_ownership: long (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- last_pymnt_amnt: double (nullable = true)
 |-- loan_status: long (nullable = true)
 |-- mo_sin_old_rev_tl_op: long (nullable = true)
 |-- mo_sin_rcnt_rev_tl_op: long (nullable = true)
 |-- mort_acc: long (nullable = true)
 |-- num_actv_rev_tl: long (nullable = true)
 |-- open_acc: long (nullable = true)
 |-- pub_rec: long (nullable = true)



In [98]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=['acc_open_past_24mths', 'annual_inc', 'avg_cur_bal', 'funded_amnt', 'grade', 'home_ownership', 'int_rate', 'last_pymnt_amnt', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mort_acc', 'num_actv_rev_tl', 'open_acc', 'pub_rec'],
    outputCol="features")

In [107]:
Final_data = Final_data.fillna(0)

In [108]:
output = assembler.transform(Final_data)

In [82]:
output.show()

+--------------------+----------+-----------+-----------+-----+--------------+--------+---------------+-----------+--------------------+---------------------+--------+---------------+--------+-------+--------------------+
|acc_open_past_24mths|annual_inc|avg_cur_bal|funded_amnt|grade|home_ownership|int_rate|last_pymnt_amnt|loan_status|mo_sin_old_rev_tl_op|mo_sin_rcnt_rev_tl_op|mort_acc|num_actv_rev_tl|open_acc|pub_rec|            features|
+--------------------+----------+-----------+-----------+-----+--------------+--------+---------------+-----------+--------------------+---------------------+--------+---------------+--------+-------+--------------------+
|                   7|   58000.0|       9536|      10400|    7|             4|    6.99|         321.08|          1|                 290|                    1|       1|              9|      17|      0|[7.0,58000.0,9536...|
|                   5|   78000.0|      29828|      15000|    5|             3|   12.39|       12017.81|         

In [109]:
# seperate the independent and target variable
(training, test)  = output.randomSplit([.7, .3])

In [52]:
# # seperate the independent and target variable
# train_x  = training.drop('loan_status')
# train_y  = training.select('loan_status')

# # seperate the independent and target variable
# test_x  = test.drop('loan_status')
# test_y  = test.select('loan_status')


In [110]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [123]:
rf = RandomForestClassifier(labelCol="loan_status", featuresCol="features", numTrees=20)

In [124]:
model = rf.fit(training)

In [125]:
predictions = model.transform(test)

In [115]:
predictions.select('prediction', 'loan_status').show()

+----------+-----------+
|prediction|loan_status|
+----------+-----------+
|       0.0|        0.0|
|       0.0|        0.0|
|       0.0|        0.0|
|       0.0|        0.0|
|       0.0|        1.0|
|       0.0|        0.0|
|       0.0|        0.0|
|       0.0|        0.0|
|       0.0|        0.0|
|       0.0|        0.0|
|       0.0|        0.0|
|       0.0|        0.0|
|       0.0|        0.0|
|       1.0|        1.0|
|       0.0|        0.0|
|       0.0|        1.0|
|       0.0|        0.0|
|       0.0|        1.0|
|       0.0|        1.0|
|       1.0|        1.0|
+----------+-----------+
only showing top 20 rows



In [116]:
model.featureImportances

SparseVector(14, {0: 0.0083, 1: 0.0042, 2: 0.0026, 3: 0.0536, 4: 0.0507, 5: 0.0003, 6: 0.1746, 7: 0.7012, 8: 0.0008, 9: 0.0002, 10: 0.0014, 11: 0.0008, 12: 0.0012, 13: 0.0})

In [126]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="loan_status", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

In [127]:
accuracy

0.8674460431654676

In [128]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="loan_status", predictionCol="prediction", metricName="f1")
f1 = evaluator.evaluate(predictions)

In [129]:
f1

0.8521044419199632